# Octopod Ensemble Model Training Pipeline

As the fourth (and final) step of this tutorial, we will train an ensemble model using the image and text models we've already trained.

This notebook was run on an AWS p3.2xlarge

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import joblib
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from transformers import AdamW, BertTokenizer, get_cosine_schedule_with_warmup

from octopod.learner import MultiTaskLearner, MultiInputMultiTaskLearner
from octopod.dataloader import MultiDatasetLoader
from octopod.ensemble import OctopodEnsembleDataset, BertResnetEnsembleForMultiTaskClassification

## Load in train and validation datasets

First we load in the csv's we created in Step 1.
Remember to change the path if you stored your data somewhere other than the default.

In [4]:
TRAIN_GENDER_DF = pd.read_csv('/home/ubuntu/fashion_dataset/gender_train.csv')

In [5]:
VALID_GENDER_DF = pd.read_csv('/home/ubuntu/fashion_dataset/gender_valid.csv')

In [6]:
TRAIN_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_train.csv')

In [7]:
VALID_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_valid.csv')

You will most likely have to alter this to however big your batches can be on your machine

In [8]:
batch_size = 128

In [9]:
bert_tok = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

max_seq_length = 128 

In [10]:
gender_train_dataset = OctopodEnsembleDataset(
    text_inputs=TRAIN_GENDER_DF['productDisplayName'],
    img_inputs=TRAIN_GENDER_DF['image_urls'],
    y=TRAIN_GENDER_DF['gender_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='train',
    crop_transform='train'

)
gender_valid_dataset = OctopodEnsembleDataset(
    text_inputs=VALID_GENDER_DF['productDisplayName'],
    img_inputs=VALID_GENDER_DF['image_urls'],
    y=VALID_GENDER_DF['gender_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='val',
    crop_transform='val'

)

season_train_dataset = OctopodEnsembleDataset(
    text_inputs=TRAIN_SEASON_DF['productDisplayName'],
    img_inputs=TRAIN_SEASON_DF['image_urls'],
    y=TRAIN_SEASON_DF['season_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='train',
    crop_transform='train'

)
season_valid_dataset = OctopodEnsembleDataset(
    text_inputs=VALID_SEASON_DF['productDisplayName'],
    img_inputs=VALID_SEASON_DF['image_urls'],
    y=VALID_SEASON_DF['season_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='val',
    crop_transform='val'
)

We then put the datasets into a dictionary of dataloaders.

Each task is a key.

In [11]:
train_dataloaders_dict = {
    'gender': DataLoader(gender_train_dataset, batch_size=batch_size, shuffle=True, num_workers=2),
    'season': DataLoader(season_train_dataset, batch_size=batch_size, shuffle=True, num_workers=2),
}
valid_dataloaders_dict = {
    'gender': DataLoader(gender_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2),
    'season': DataLoader(season_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2),
}

In [12]:
TrainLoader = MultiDatasetLoader(loader_dict=train_dataloaders_dict)
len(TrainLoader)

366

In [13]:
ValidLoader = MultiDatasetLoader(
    loader_dict=valid_dataloaders_dict,
    shuffle=False
)
len(ValidLoader)

123

Create Model and Learner
===

Since the image model could potentially have multiple Resnets for different subsets of tasks, we need to create an `image_task_dict` that splits up the tasks grouped by the Resnet they use.

This version uses the same resnet for gender and season, but we could just as easily have trained separate models for each task.

In [14]:
image_task_dict = {
    'gender_season': {
        'gender': TRAIN_GENDER_DF['gender_cat'].nunique(),
        'season': TRAIN_SEASON_DF['season_cat'].nunique()
    }  
}

We still need to create the `new_task_dict` for the learner.

In [15]:
new_task_dict = {
    'gender': TRAIN_GENDER_DF['gender_cat'].nunique(),
    'season': TRAIN_SEASON_DF['season_cat'].nunique()
}

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


We first initialize the model by setting up the right shape with the image_task_dict.

In [17]:
model = BertResnetEnsembleForMultiTaskClassification(
    image_task_dict=image_task_dict
)

We then load in the existing models by specifying the folder where the models live and their id's.

In [18]:
resnet_model_id_dict = {
    'gender_season': 'IMAGE_MODEL1'
}

In [19]:
model.load_core_models(
    folder='/home/ubuntu/fashion_dataset/models/',
    bert_model_id='TEXT_MODEL1',
    resnet_model_id_dict=resnet_model_id_dict
)

We've set some helper methods that will freeze the core bert and resnets for you if you only want to train the new layers. As with all other aspects of training, this is likely to require some experimentation to determine what works for your problem.

You will likely need to explore different values in this section to find some that work
for your particular model.

In [20]:
model.freeze_bert()
model.freeze_resnets()

loss_function = nn.CrossEntropyLoss()

lr_last = 1e-3
lr_main = 1e-5

lr_list = [
    {'params': model.bert.parameters(), 'lr': lr_main},
    {'params': model.dropout.parameters(), 'lr': lr_main},   
    {'params': model.image_resnets.parameters(), 'lr': lr_main},
    {'params': model.image_dense_layers.parameters(), 'lr': lr_main},
    {'params': model.ensemble_layers.parameters(), 'lr': lr_last},
    {'params': model.classifiers.parameters(), 'lr': lr_last},
]

optimizer = optim.Adam(lr_list)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size= 4, gamma= 0.1)

In [21]:
learn = MultiInputMultiTaskLearner(model, TrainLoader, ValidLoader, new_task_dict)

Train Model
===

As your model trains, you can see some output of how the model is performing overall and how it is doing on each individual task.

In [22]:
learn.fit(
    num_epochs=10,
    loss_function=loss_function,
    scheduler=exp_lr_scheduler,
    step_scheduler_on_batch=False,
    optimizer=optimizer,
    device=device,
    best_model=True
)

train_loss,val_loss,gender_train_loss,gender_val_loss,gender_acc,season_train_loss,season_val_loss,season_acc,time
0.337576,0.346522,0.060817,0.047060,0.987166,0.706797,0.745998,0.701307,04:32
0.300118,0.337571,0.034276,0.032321,0.990093,0.654773,0.744771,0.700405,04:33
0.294485,0.315664,0.031841,0.033477,0.990431,0.644874,0.692097,0.723082,04:33
0.289034,0.353571,0.029288,0.031247,0.990656,0.635557,0.783547,0.692296,04:33
0.280033,0.302999,0.024496,0.026899,0.991332,0.620940,0.671313,0.723382,04:33
0.278021,0.296622,0.023456,0.025382,0.991782,0.617632,0.658451,0.732542,04:33
0.276779,0.288762,0.022753,0.024146,0.991782,0.615671,0.641755,0.739601,04:33
0.276053,0.292702,0.022104,0.025689,0.991895,0.614843,0.648892,0.738099,04:33
0.272742,0.293113,0.021063,0.025314,0.991444,0.608504,0.650351,0.738700,04:33
0.272881,0.291344,0.022217,0.024019,0.991895,0.607287,0.647951,0.740201,04:33


Epoch 6 best model saved with loss of 0.2887622564998693


The ensemble model performs better on both the gender and season tasks than either the image or text model alone.

Checking validation data
===

We provide a method on the learner called `get_val_preds`, which makes predictions on the validation data. You can then use this to analyze your model's performance in more detail.

In [24]:
pred_dict = learn.get_val_preds(device)

In [25]:
pred_dict

{'gender': {'y_true': array([0., 2., 4., ..., 4., 2., 2.]),
  'y_pred': array([[9.79963720e-01, 8.20267305e-05, 1.99036356e-02, 3.73025723e-05,
          1.32717532e-05],
         [3.16389269e-05, 1.57239319e-05, 9.99903560e-01, 2.93510820e-05,
          1.97479749e-05],
         [5.46117808e-05, 2.69602460e-04, 3.34556389e-05, 8.94905243e-05,
          9.99552786e-01],
         ...,
         [3.90316336e-06, 2.12771658e-04, 6.02410091e-06, 2.43015779e-06,
          9.99774754e-01],
         [5.30419175e-06, 4.19809066e-06, 9.99900460e-01, 2.53984490e-05,
          6.46425178e-05],
         [8.87045371e-06, 2.76470905e-06, 9.99982119e-01, 4.77818230e-06,
          1.44818603e-06]])},
 'season': {'y_true': array([0., 0., 2., ..., 2., 3., 3.]),
  'y_pred': array([[8.35114121e-01, 1.27773534e-03, 1.58392146e-01, 5.21592796e-03],
         [1.97143480e-01, 2.40618410e-03, 7.77141452e-01, 2.33088844e-02],
         [9.69123393e-02, 1.56298105e-03, 8.78993690e-01, 2.25310437e-02],
         ...

Save/Export Model
===

The ensemble model can also be saved or exported.

In [ ]:
model.save(folder='/home/ubuntu/fashion_dataset/models/', model_id='ENSEMBLE_MODEL1')

In [29]:
model.export(folder='/home/ubuntu/fashion_dataset/models/', model_id='ENSEMBLE_MODEL1')